Testing code to create grid coordinates for a scan for four inputted coordinates. 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import functools
import math

In [ ]:
#Example points and step size a user might plug in; can be any number of x,y coordinates
#FIXME: Needs to be a convex shape (breaks for concave)
pts = [[0,0.5],[3,2],[2,0.5],[0,1],[2,-1],[-0.5,0.5]]
pts3d = [[0,0.5,0],[3,2,1],[2,0.5,1.5],[0,1,0.5],[2,-1,-1],[-0.5,0.5,0]]
step = 0.1

#Define a measure of angle to sort coordinates into clockwise list
def clockwiseangle_and_distance(point):
    origin = [0,0]
    refvec = [0,1]  #i.e., pointing up
    
    # Vector between point and the origin: v = p - o
    vector = [point[0]-origin[0], point[1]-origin[1]]
    
    # Length of vector: ||v||
    lenvector = math.hypot(vector[0], vector[1])
    
    # If length is zero there is no angle
    if lenvector == 0:
        return -math.pi, 0
    
    # Normalize vector: v/||v||
    normalized = [vector[0]/lenvector, vector[1]/lenvector]
    dotprod  = normalized[0]*refvec[0] + normalized[1]*refvec[1]     # x1*x2 + y1*y2
    diffprod = refvec[1]*normalized[0] - refvec[0]*normalized[1]     # x1*y2 - y1*x2
    angle = math.atan2(diffprod, dotprod)
    
    # Negative angles represent counter-clockwise angles so we need to subtract them 
    # from 2*pi (360 degrees)
    if angle < 0:
        return 2*math.pi+angle, lenvector
    
    #FIXME: if two vectors have the same angle then the shorter distance should come first.
    return angle, lenvector

sorted_pts = sorted(pts3d, key = clockwiseangle_and_distance)
sorted_pts.append(sorted_pts[0])   #add starting coordinate to the end to make a closed shape in plot
arr = np.array(sorted_pts)

In [ ]:
#Generate grid that covers shape from min to max values
x = np.arange(min(arr[:,0]), max(arr[:,0])+step, step)  #add step to include max
y = np.arange(min(arr[:,1]), max(arr[:,1])+step, step)
z = np.arange(min(arr[:,2]), max(arr[:,1])+step, step)
#a,b = np.meshgrid(x,y)
a,b,c = np.meshgrid(x,y,z)
#points = np.stack([a.reshape(-1), b.reshape(-1)]).T     #reshape into array
points3d = np.stack([a.reshape(-1), b.reshape(-1),c.reshape(-1)]).T     #reshape into array

print(points3d)

In [ ]:
#METHOD 1-- Doesn't work on concave shapes, great for convex, any number of vertices

#Calculate center of array
center = np.mean(arr, axis = 0)

# #Create true/false array to mask points covering shape
def normal(a,b):
    v = b-a
    n = np.array([v[1], -v[0]]) #FIXME: problem with representing in 3d is here
    #normal needs to point out
    if (center-a)@n > 0:
         n *= -1
    return n

mask = functools.reduce(np.logical_and, [((points3d-a)@normal(a, b)) > 0.001 for a,b in zip(arr[:-1], arr[1:])])
#mask2 = functools.reduce(np.logical_and, [((points-a)@normal(a, b)) < 0 for a,b in zip(arr[:-1], arr[1:])])
plt.plot(arr[:,0],arr[:,1],arr[:,2])
plt.gca().set_aspect('equal')
plt.scatter(points3d[mask][:,0], points3d[mask][:,1],points3d[mask][:,2])
#plt.scatter(points[mask][:,0], points[mask][:,1])
#plt.scatter(points[:,0], points[:,1])

In [14]:
for a,b in zip(arr[:-1], arr[1:]):
    

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 3)

In [ ]:
#TODO: test in 3D

from matplotlib.path import Path

x = np.arange(min(arr[:,0]), max(arr[:,0])+step, step)  #add step to include max
y = np.arange(min(arr[:,1]), max(arr[:,1])+step, step)
a,b = np.meshgrid(x,y)
points = np.stack([a.reshape(-1), b.reshape(-1)]).T 

# x, y = np.meshgrid(np.arange(300), np.arange(300)) # make a canvas with coordinates
# x, y = x.flatten(), y.flatten()
# points = np.vstack((x,y)).T 

p = Path(arr) # make a polygon
grid = p.contains_points(points)
#mask = grid.reshape(300,300) # now you have a mask with points inside a polygon

#ax = plt.figure().add_subplot(projection = '3d')

plt.plot(arr[:,0],arr[:,1],arr[:,2])
plt.gca().set_aspect('equal')
plt.scatter(points[grid][:,0], points[grid][:,1],points3d[grid][:,2])

#plt.show()

2.28.24 -- Traditional grid scans, beginning with what kind of inputs. Later: addressing slow motor/nested scanning. 

In [23]:
def scan_layer(
    start_pos = None,
    nsteps = None,
    step = None, 
    end_pos = None,
    motor = None
):
    return start_pos, nsteps, motor



In [28]:
my_start, my_steps, my_motor = scan_layer(1,2,3,5)

In [29]:
my_motor